<div style="width: 100%; clear: both;">
    <div style="float: left; width: 50%;">
        <img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg" , align="left">
    </div>
</div>
<div style="float: right; width: 50%;">
    <p style="margin: 0; padding-top: 22px; text-align:right;">22.521 · Análisis de redes sociales</p>
    <p style="margin: 0; text-align:right;">Grado de Ciencia de Datos Aplicada (<i>Applied Data Science</i>)</p>
    <p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
<div style="width: 100%; clear: both;"></div>
<div style="width:100%;">&nbsp;</div>

# Extracción de datos: el caso de Twitter y su API

En este material vamos a ver un caso particular de extracción de datos de una red social: Twitter. 

En primer lugar, necesitaremos las claves de desarrolador para poder aceder a la API de Twitter. Si ya las habéis solicitado, las podréis obtener desde vuestra cuenta. En caso contrario, visitad https://developer.twitter.com/en/apply-for-access.html para obtener información y solicitar acceso las claves.

La clave de API de Twitter está, en realidad, formado por cuatro claves. Cuando tengáis cuenta de desarrollador, basta con acceder al menú **Apps** > **Create an app** y crear una *app* nueva. Dentro de sus detalles encontraréis **Keys and tokens**. Ahí se pueden crear y regenerar las claves.

Introducid vuestras claves API en la primera celda. Recordad que no se deben compartir, para evitar el uso fraudulento y también para evitar superar los límites de la API.

In [ ]:
####input your credentials here
consumer_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
consumer_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
access_token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
access_token_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

Aunque podríamos hacer llamadas directamente a la API, una de las mejores formas es aprovechar alguna librería adicional para que gestione algunos temas por nosotros y que nos simplifique las cosas. En este caso, vamos a usar `Tweepy` (http://docs.tweepy.org/en/v3.8.0/), aunque existen otras igualmente válidas como `Twython` (podéis usar la que prefiráis, si conocéis otras librerías). Vamos a importarlo.

In [ ]:
import tweepy

Un ejemplo sencillo (el "hola mundo" de Tweepy) consiste en descargar los Tweets de nuestro propio TL (*timeline*). Esto incluye a las cuentas que seguís. Si no tenéis ninguno, escribid uno. Poned un texto genérico "Hola mundo", por ejemplo.

In [ ]:
# Preparamos la autenticación
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Preparamos el módulo api de Tweepy (que es el que nos va a ayudar a hacer las "preguntas" a la API)
api = tweepy.API(auth)

# Y hacemos la primera llamada: fijaros que api.home_timeline() es una función que devuelve los tweets de nuestro propio 
# timeline (del usuario cuyas claves son usadas)
public_tweets = api.home_timeline()

In [ ]:
# Lo que devuelve la llamada es un conjunto de tweets que se pueden iterar. Vamos a ver sus textos, por ejemplo.
for tweet in public_tweets:
    print(tweet.text)

## Check

Podéis comprobar como coinciden con lo que veis en el cliente web de Twitter o en su app móvil.

# Recorrer y sacar información de los Tweets

Cada objeto que se obtiene de `Tweepy` tiene varios atributos. Antes hemos visto que con ".text" obtenemos el texto del tweet y, si buscáis en la documentación, veréis muchos más. Pero el que nos va a ser más útil aquí va a ser usar directamente el JSON (JSON es un formato estructurado de datos) en bruto, accesible a partir de. "._json".

Por ejemplo, imaginad que queréis ver los nombres de los autores de los últimos tweets de vuestro TL:

In [ ]:
for tweet in public_tweets:
    print(tweet._json['user']['screen_name'])

Fijaros que el acceso es como un diccionario, una forma que ya habéis visto en el pasado. Pero... ¿por qué aunque mi TL tenga centenares o miles de mensajes solo estoy viendo los últimos 20? Aquí está una de las partes más relevantes de todo esto.

## Límites

La API de Twitter tiene límites, más laxos en algunos casos, más estrictos en otros. Tenéis la referencia a todas las funciones de la API en https://developer.twitter.com/en/docs/api-reference-index, pero para la explicación nos centraremos en dos: Search y Friends.

En Search (https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets ), encontraréis los detalles de los parámetros para hacer las llamadas, junto a un ejemplo y una respuesta de ejemplo. A esto nos ayudará Tweepy, pero deberíais fijaros un poco más arriba, en la parte de "Resource information". Hay un apartado que pone "Rate Limited?" (Yes), lo que significa que tiene máximos de preguntas. Y más abajo, "Requests/15-min window (app auth)" (450). Esto lo que significa es, básicamente, que cada 15 minutos solo podéis hacer 450 preguntas de búsqueda. Parece mucho pero no lo es; cada "pregunta" devuelve como mucho 100 tweets (o 15 si no especificamos nada), con lo que obtener datos sobre un hashtag o palabra popular puede consumir fácilmente las 450 llamadas. Además, la parte gratuita solo da información de búsqueda de los últimos 7 días, así que hay que tenerlo en cuenta.

En Friends, que es otra que podría ser interesante (para ver a quién sigue un usuario concreto), encontramos un problema grave. Fijaros que solo permite ¡15! llamadas cada 15 minutos. Es decir, en una red pequeña, de 1500 usuarios, estaríamos unas 25 horas para sacar toda la información si no fallase nada. La realidad es que, probablemente, estemos día y medio o dos días enteros. Tenedlo en cuenta si planeáis usar esta (interesante, eso sí), relación. 

Normalmente manejaríamos esto desde Python. Haríamos una llamada a la API y nos devolvería un valor (https://developer.twitter.com/en/docs/basics/response-codes ). "200" es el indicador de que todo está bien, pero si devuelve otro código es que hay un error. "429", por ejemplo, es "Too many requests", indicador de haber excedido el límite. Esto lo procesaríamos como una excepción y, de entrar en ello, esperaríamos 15 minutos antes de volver a llamar (o un tiempo prudencial). La suerte es que Tweepy lo puede hacer por nosotros así que, para el objetivo de este tutorial, no hace falta preocuparse en exceso por ello (más allá de tenerlo en cuenta como limitación de extracción).

## Búsqueda

Vamos a hacer un ejemplo de búsqueda con Tweepy, que no tan solo maneja los tiempos de espera sino también las extracciones mayores que el tamaño máximo de búsqueda - usando un invento llamado cursor. Funciona así:

In [ ]:
# Hagamos una búsqueda de Tweets con la palabra #Barcelona o #Madrid

# Se pueden usar operadores binarios (más detalle en la documentación de la API)
for tweet in tweepy.Cursor(api.search,q=("#Barcelona OR #Madrid"),count=100,
                           tweet_mode= 'extended').items(200):
    print(tweet._json['full_text'])
    if 'retweeted_status' in tweet._json:
        print("Es un RT y su texto completo es: " + tweet._json['retweeted_status']['full_text'])

# Acabamos de pedir los 200 tweets más recientes, devueltos en páginas de 100 (que es el máximo)
# Con las palabras #Barcelona o #Madrid
# tweet_mode = extended se usa porque desde que Twitter alargó los tweets (de 140 a 280 carácteres)
# se almacena el tweet de 280 carácteres en otro atributo; el "text" normal estaria truncado.

Fijaros en las diferencias entre campos y en el acceso dentro de los RT. Podéis jugar con ello e investigar. 

## Problema: ¿Y el almacenamiento?

Hasta este punto, cada vez que buscamos un conjunto de tweets en las instrucciones anteriores, estos se renuevan o desaparecen. Así que interesa hacer una extracción y conservarla. Aquí hay dos estrategias básicas:

- O bien se conservan los campos que sean relevantes para el ejercicio en un fichero estructurado (por ejemplo, en un CSV que tenga por filas los usuarios y sus interacciones).

- O bien se usa una base de datos para el almacenamiento. Aquí se recomienda usar MongoDB, que además tiene una librería muy sencilla para Python llamada `pymongo`.

Ambas opciones pueden ser válidas.

La opción más sencilla es un CSV y se puede hacer de múltiples formas al gusto del consumidor. Aquí un ejemplo, aunque normalmente guardaríamos más campos (usuario al que se hace RT en caso de serlo, a quién se responde, etc.). Notad que si usáis un CSV hay que contemplar todos los casos y preparar tokens o comodines para campos vacíos (e.g. si un Tweet no es un RT, poner un "*None*" en ese campo). Cada fila debe contener el mismo número de elementos, es importante.

In [ ]:
# Guardamos con un CSV fila por fila, ojo que los carácteres separadores nos pueden jugar malas pasadas
import csv

with open('tweet_list.csv', 'w') as csvfile:
            writer = csv.writer(csvfile, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            for tweet in tweepy.Cursor(api.search,q=("#Barcelona OR #Madrid"),count=100,
                           tweet_mode= 'extended').items(200):
                retweet = False
                if hasattr(tweet, 'retweeted_status'):
                    retweet = True
                writer.writerow([tweet._json['user']['id'], tweet._json['user']['screen_name'], tweet._json['full_text']])

In [ ]:
# Y para una lectura rápida y estructurada, pandas:
import pandas as pd

df = pd.read_csv('tweet_list.csv', header=None, sep=";")
df.head(10)

## Para MongoDB:

- Instalación: https://docs.mongodb.com/manual/administration/install-community/
- Inicio: https://docs.mongodb.com/manual/tutorial/install-mongodb-on-windows/#run-mdb-edition-from-the-command-interpreter (hay que recordar iniciar el servicio mongod siempre que se quiera usar).
- GUI (Compass): https://www.mongodb.com/download-center/compass

In [ ]:
# Ejemplo MongoDB

# Importamos PyMongo
from pymongo import MongoClient

# Creamos una conexión con la BBDD
client = MongoClient()
           
# Usamos una base de datos llamada test
db = client.test

# Y cada Tweet es almacenado en una colección llamada "tweets"
for tweet in tweepy.Cursor(api.search,q=("#Barcelona OR #Madrid"),count=100,
                           tweet_mode= 'extended').items(200):
    db.tweets.insert_one(tweet._json)

## Recuperación de MongoDB

Ahora ya podríamos trabajar, cuando quisiéramos, con lo que se ha recogido en la BBDD de una forma bien directa: nuestra BBDD pasaría a ser la fuente de tweets. Fijaros en su uso base:

In [ ]:
import networkx as nx

G = nx.DiGraph()

for result in db.tweets.find():
            uid = result['user']['screen_name']
            G.add_node(uid) # si ya existe lo omite solo
            
            # Hay más casuísticas, pero por ejemplo, con el retweeted_status (es decir, un user ha RT a otro)
            if 'retweeted_status' in result:
                if G.has_edge(uid, result['retweeted_status']['user']['screen_name']):
                    G[uid][result['retweeted_status']['user']['screen_name']]['weight'] += 1.0
                else:
                    G.add_edge(uid, result['retweeted_status']['user']['screen_name'], weight = 1.0) 

Podéis usar Compass para ver el contenido de la BBDD. 

# *Streaming*

Muchas veces, no obstante, en las plataformas online nos interesa "consumir" información al momento. Si antes hemos visto la búsqueda (a toro pasado) ahora vamos a ver el streaming en tiempo real.

https://developer.twitter.com/en/docs/tweets/filter-realtime/overview

Una consideración importante: si usáis *streaming* para una investigación tiene un problema de base. Twitter afirma que solo nos proporciona un muestreo de, aproximadamente, el 1% de los tweets que circulan en cualquier momento y lo hace según su propio algoritmo de relevancia. Así que lo que obtenéis es un filtro sobre una muestra filtrada por la relevancia según Twitter. Esto podría ser bueno pero la realidad es que, como veréis, a veces se cuela mucho *spam* tuitero.

¿Cómo preparamos un Stream con Tweepy? Conceptualmente es distinto de lo que hemos visto hasta ahora, porque en realidad lo que haremos es preparar un hilo que dejará nuestro programa "escuchando" del Stream de Twitter y cada vez que llegue un *tweet* que coincida con nuestro filtro saltará a una rutina para tratarlo (por ejemplo, para almacenarlo o escribirlo) y seguirá escuchando. Hay que mantener esa rutina al mínimo para evitar que interfiera con el siguiente *tweet* (aunque se ponga a una cola, si se llena el *pipeline*, la conexión se rompe automáticamente).

In [ ]:
# Para trabajar con json's
import json

# Pondríamos la lista de palabras
WORDS = ['#barcelona', '#madrid']

# Repito las claves
consumer_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
consumer_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
access_token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
access_token_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

# Crear la clase que realizará la escucha
class StreamListener(tweepy.StreamListener):    
    #Esta es la clase que usa tweepy para acceder a la API de Streaming. 

    def on_connect(self):
        # Para conectar a la Streaming API
        print("You are now connected to the streaming API.")
 
    def on_error(self, status_code):
        # Si hay error no desconecta (o no tiene por qué), pero muestra el error
        print('An Error has occured: ' + repr(status_code))
        return False
 
    def on_data(self, data):
        # Aquí está lo principal, el tratamiento al recibir "data" (un tweet)
        # Podríamos, por ejemplo, conectar al mongoDB y almacenar el tweet
        try:
            
            # Dejo el código de ejemplo, siempre recordad que los imports los tenemos en código anterior.
            '''client = MongoClient()
            db = client.testserver
            datajson = json.loads(data)
            db.tweets.insert_one(datajson)'''
            
            # Codificamos el JSON a un formato python
            datajson = json.loads(data)
            
            # Cogemos el texto del tweet, por ejemplo, así vemos qué estamos cogiendo
            text = datajson['text']
            fecha = datajson['created_at']

            # E imprimimos un mensaje conforme acabamos de capturar un Tweet            
            print("Tweet capturado a las "  + str(fecha) + " con el texto " + str(text))
           
        except Exception as e:
            print(e)

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Aquí simplemente preparamos la escucha (listener)
# Necesitamos poner el 'wait_on_rate_limit=True' para gestionar las limitaciones de la API de Twitter.
listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True)) 
streamer = tweepy.Stream(auth=auth, listener=listener)

# Imprimir lo que estamos buscando
print("Tracking: " + str(WORDS))
streamer.filter(track=WORDS)

Los *streams* no tienen "fin". Hay que pararlos de forma manual y reiniciar el código entero si queremos volver a activarlo y no tener problemas. 